In [0]:
spark.conf.set("fs.azure.account.auth.type.spotifyadls.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.spotifyadls.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.spotifyadls.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-06-18T09:16:14Z&st=2024-06-16T01:16:14Z&spr=https&sig=sbia7gRMs55MIObCjvK9XRx3VH6wcCc%2BpB%2F5OuijO8Y%3D")




In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

df = spark.read.format("csv").option("header","true").load("abfss://raw-data@spotifyadls.dfs.core.windows.net/spotify-2023.csv")

In [0]:
df.limit(10).toPandas()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,263,45,10,826,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,126,58,14,382,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,207,91,14,949,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,207,125,12,548,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,133,87,15,425,144,A,Minor,65,23,80,14,63,11,6
5,Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,183706234,67,213,88,17,946,141,C#,Major,92,66,58,19,0,8,24
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,725980112,34,222,43,13,418,148,F,Minor,67,83,76,48,0,8,3
7,Columbia,Quevedo,1,2023,7,7,714,43,58149378,25,89,30,13,194,100,F,Major,67,26,71,37,0,11,4
8,fukumean,Gunna,1,2023,5,15,1096,83,95217315,60,210,48,11,953,130,C#,Minor,85,22,62,12,0,28,9
9,La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,553634067,49,110,66,13,339,170,D,Minor,81,56,48,21,0,8,33


In [0]:
df = df.withColumn(
    'release_date',
    to_date(concat(
        col('released_year').cast('string'), lit('-'),
        col('released_month').cast('string'), lit('-'),
        col('released_day').cast('string')
    ))
)

# Drop the original year, month, and day columns
df = df.drop('released_year', 'released_month', 'released_day', 'bpm', 'key', 'mode')

# Show the resulting DataFrame
df.show()

+--------------------+--------------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|          track_name|      artist(s)_name|artist_count|in_spotify_playlists|in_spotify_charts|   streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|release_date|
+--------------------+--------------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|Seven (feat. Latt...|    Latto, Jung Kook|           2|                 553|              147

In [0]:
from pyspark.sql.types import IntegerType

df = df.withColumn('streams', df['streams'].cast(IntegerType()))


In [0]:
df = df.orderBy(col('streams').desc())

# Drop duplicates based on 'track_name', keeping the first occurrence (highest streams)
df = df.dropDuplicates(['track_name'])


In [0]:
# Check for null values and sum them up
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()


+----------+--------------+------------+--------------------+-----------------+-------+------------------+---------------+-------------------+----------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|track_name|artist(s)_name|artist_count|in_spotify_playlists|in_spotify_charts|streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|release_date|
+----------+--------------+------------+--------------------+-----------------+-------+------------------+---------------+-------------------+----------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|         0|             0|           0|                   0|                0|     26|                 0|              0|                  0|         

In [0]:
df = df.drop('in_shazam_charts')


In [0]:
df.printSchema()

root
 |-- track_name: string (nullable = true)
 |-- artist(s)_name: string (nullable = true)
 |-- artist_count: string (nullable = true)
 |-- in_spotify_playlists: string (nullable = true)
 |-- in_spotify_charts: string (nullable = true)
 |-- streams: integer (nullable = true)
 |-- in_apple_playlists: string (nullable = true)
 |-- in_apple_charts: string (nullable = true)
 |-- in_deezer_playlists: string (nullable = true)
 |-- in_deezer_charts: string (nullable = true)
 |-- danceability_%: string (nullable = true)
 |-- valence_%: string (nullable = true)
 |-- energy_%: string (nullable = true)
 |-- acousticness_%: string (nullable = true)
 |-- instrumentalness_%: string (nullable = true)
 |-- liveness_%: string (nullable = true)
 |-- speechiness_%: string (nullable = true)
 |-- release_date: date (nullable = true)



In [0]:
# Define columns to convert to integer (columns 2 to 16, excluding column 5)
columns_to_convert = [col_name for col_name in df.columns[2:16] if col_name != 'streams']

# Convert selected columns to integer type
for col_name in columns_to_convert:
    df = df.withColumn(col_name, col(col_name).cast('integer'))

# Show the resulting DataFrame
df.show()

+--------------------+--------------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|          track_name|      artist(s)_name|artist_count|in_spotify_playlists|in_spotify_charts|   streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|release_date|
+--------------------+--------------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+--------------+---------+--------+--------------+------------------+----------+-------------+------------+
|"Apna Bana Le (Fr...|Arijit Singh, Sac...|           2|                  86|                0| 139836056|                11|            101|    

In [0]:
df.write.option("header", True).mode("overwrite").csv("abfss://processed-data@spotifyadls.dfs.core.windows.net/")